In [5]:
import sys
import os
import csv
from tqdm import tqdm_notebook
import pandas
import numpy
from pandas.tseries.holiday import USFederalHolidayCalendar
from statsmodels.tsa.arima_model import ARIMA
import sklearn.model_selection

import warnings
warnings.simplefilter('once', category=UserWarning)

os.chdir('/home/chlang/Projects/prognosec')

sys.path.append('/home/chlang/Projects/prognosec/prognosec')

%load_ext autoreload
%autoreload 1
%aimport models
%aimport timeseries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
with open('data/data-1574048454921.csv', 'r', encoding='utf-8') as f:
    csvreader = csv.DictReader(f, delimiter=',')

    ts_ed = [row for row in csvreader]

    ts_ed = pandas.DataFrame.from_dict(ts_ed).drop(['frequency', 'intraperiod'], axis=1)
    ts_ed['date'] = pandas.to_datetime(ts_ed['date'], format='%Y-%m-%d')
    ts_ed = ts_ed.set_index('date').drop('secid', 1).apply(pandas.to_numeric)

    ts_ed = timeseries.Timeseries(ts_ed, 'adj_close', timeseries.CAL_NYSE)
    
with open('data/data-1574048640489.csv', 'r', encoding='utf-8') as f:
    csvreader = csv.DictReader(f, delimiter=',')

    ts_aapl = [row for row in csvreader]

    ts_aapl = pandas.DataFrame.from_dict(ts_aapl).drop(['frequency', 'intraperiod'], axis=1)
    ts_aapl['date'] = pandas.to_datetime(ts_aapl['date'], format='%Y-%m-%d')
    ts_aapl = ts_aapl.set_index('date').drop('secid', 1).apply(pandas.to_numeric)

    ts_aapl = timeseries.Timeseries(ts_aapl, 'adj_close', timeseries.CAL_NASDAQ)

In [7]:
# ts_aapl.transformations.add('volume', numpy.log10)
# ts_aapl.transformations.add('adj_open', numpy.sqrt)
# ts_aapl.enable_transforms = True

In [8]:
tmp = models.RandomWalk(ts=ts_aapl)

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [9]:
help(tmp)

Help on RandomWalk in module models object:

class RandomWalk(Arima)
 |  RandomWalk(ts: Union[prognosec.timeseries.Timeseries, NoneType] = None)
 |  
 |  Autoregressive integrated moving average (ARIMA) time series model
 |  
 |  Implements the non-seasonal ARIMA time series model for forecasting
 |  purposes. The main parameters is the (p, d, q) order:
 |  - p : Number of time lags (order of autoregressive model)
 |  - d : Degree of differencing
 |  - q : Order of moving average
 |  
 |  Model is provided by the `statsmodels` library
 |  
 |  Attributes
 |  ----------
 |  ts : timeseries.Timeseries
 |  pseries : str
 |  params : ModelParameters
 |  param_names : array_like[str]
 |  
 |  Parameters
 |  ----------
 |  order : array_like[int]
 |      The (p, d, q) order of the ARIMA model
 |  ts : timeseries.Timeseries
 |      The timeseries dataset containing the time series for forecasting
 |      and exogenous/independent variables
 |  
 |  Method resolution order:
 |      RandomWalk


In [10]:
orders = [(0, 1, 0), (1, 1, 0), (0, 1, 1), (1, 0, 0)]

In [11]:
fit_results = list()
for an_order in tqdm_notebook(orders):
    r = models.cv_timeseries(ts_aapl, models.Arima, order=an_order)
    fit_results.append(r)
    
final_fit_result = pandas.concat(fit_results)

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:221: RuntimeWarning:

divide by zero encountered in true_divide

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available

/home/chlang/anaconda3/envs/promometheus/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning:

Inverting hessian failed, no bse or cov_params available



In [13]:
final_fit_result

,k,series,model,train_size,test_size,mse,var,AIC,BIC
0,0,adj_close,"ARIMA(0, 1, 0)",897,891,0.196612,1.303324e-01,-5038.000854,-5028.404973
1,1,adj_close,"ARIMA(0, 1, 0)",1788,891,0.053166,6.840522e-03,-8707.529826,-8696.553239
2,2,adj_close,"ARIMA(0, 1, 0)",2679,891,0.194312,3.359369e-02,-12013.025046,-12001.239395
3,3,adj_close,"ARIMA(0, 1, 0)",3570,891,0.383900,8.823094e-02,-15340.151751,-15327.791669
4,4,adj_close,"ARIMA(0, 1, 0)",4461,891,1.349203,5.282433e+00,-19256.104373,-19243.298565
5,5,adj_close,"ARIMA(0, 1, 0)",5352,891,2.693832,2.378060e+01,-16871.614513,-16858.444435
6,6,adj_close,"ARIMA(0, 1, 0)",6243,891,74.336862,8.124524e+03,-18993.429601,-18979.951489
7,7,adj_close,"ARIMA(0, 1, 0)",7134,891,1106.949577,1.722022e+06,-6422.102471,-6408.357497
8,8,adj_close,"ARIMA(0, 1, 0)",8025,891,385.684316,1.056341e+05,3123.447793,3137.428177
9,9,adj_close,"ARIMA(0, 1, 0)",8916,891,4290.978123,1.713345e+07,14097.815007,14112.005988
